In [ ]:
## reset QDAC to 0V
# also set CH6 for LA Q2 to sweet spot
import qdac
import random
import math

with qdac.qdac('COM3') as q:
    print("QDAC Serial number: %s" % q.getSerialNumber())
    print("Number of channels: %d" % q.getNumberOfChannels())

    print("-----------------------------------------------")
    print("Setting All Channels DC voltage to 0 V")
    for n in range(1,24,1):
        # print("Setting Channel %d voltage range to 10 V" % n)
        # result = q.setVoltageRange(channel=n, theRange=10)
        # print("Result: %s" % result)
        #print("Setting Channel %d DC voltage to 0 V" % n)
        result = q.setDCVoltage(channel=n, volts=0)
        #print("Result: %s" % result)
        #voltage1 = q.getDCVoltage(n)
        #print("Voltage output on channel %d is %f V" % n % voltage1)

    print("Setting Channel 6 DC voltage to sweet spot")
    result = q.setDCVoltage(channel=6, volts=6.4352E-3)

In [1]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm import SimulationConfig, LoopbackInterface
from configuration import *
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from qualang_tools.loops import from_array
%matplotlib qt

# Start to save data
import datetime 
import os
import pandas as pd 

now = datetime.datetime.now()
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")
second  = now.strftime("%S")

tPath = os.path.join(r'Z:\LabberData_DF5\QM_Data_DF5',year,month,'Data_'+month+day)
if not os.path.exists(tPath):
   os.makedirs(tPath)
exp_name = 'Rabi_chevron'
qubit_name = 'LA_Q2'

f_str =exp_name + '_'+ qubit_name +'_'+ hour+ '_' + minute +'_' + second
f_name= f_str+'.csv'
j_name = f_str + '_state.json'
p_name= f_str+'.png'

# End to save data

#########################################
# Set-up the machine and get the config #
#########################################
machine = QuAM("quam_state.json", flat_data=False)
# machine.qubits[0].pi_amp=0.309
config = build_config(machine)

###################
# The QUA program #
###################

n_avg = 200  # Number of averaging loops
cooldown_time = 20_000 * u.ns
qubit_index = 0
number_of_qubits = 1
qb_list = [i for i in range(number_of_qubits)]
qb_if_list = [machine.qubits[i].f_01 - machine.qubits[i].lo for i in range(number_of_qubits)]

# Frequency sweep in Hz (Needs to be a list of int)
dfs = np.arange(-50e6, +25e6, 0.5e6)

# Pulse amplitude sweep (as a pre-factor of the flux amplitude)
amps = np.arange(0, 1.99, 0.1)

###################
# The QUA program #
###################

with program() as rabi_amp_freq:
    n = declare(int)  # Averaging index
    f = declare(int)  # Resonator frequency
    a = declare(fixed)  # Pulse amplitude
    I = declare(fixed)
    Q = declare(fixed)
    n_st = declare_stream()
    I_st = declare_stream()
    Q_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        with for_(*from_array(a, amps)):
            with for_(*from_array(f, dfs)):
                # Update the qubit frequency
                update_frequency(machine.qubits[qubit_index].name, f + qb_if_list[qubit_index])
                # Adjust the pulse amplitude
                play("x180" * amp(a), machine.qubits[qubit_index].name)
                align(machine.qubits[qubit_index].name, machine.resonators[qubit_index].name)
                # Measure the resonator
                measure(
                    "readout",
                    machine.resonators[qubit_index].name,
                    None,
                    dual_demod.full("cos", "out1", "sin", "out2", I),
                    dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
                )
                # Wait for the resonator to cooldown
                wait(cooldown_time, machine.resonators[qubit_index].name)
                # Save data to the stream processing
                save(I, I_st)
                save(Q, Q_st)
        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(dfs)).buffer(len(amps)).average().save("I")
        Q_st.buffer(len(dfs)).buffer(len(amps)).average().save("Q")
        n_st.save("iteration")


#####################################
#  Open Communication with the QOP  #
#####################################
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

simulation = False
if simulation:
    simulation_config = SimulationConfig(
        duration=28000, simulation_interface=LoopbackInterface([("con1", 3, "con1", 1)])
    )
    job = qmm.simulate(config, rabi_amp_freq, simulation_config)
    job.get_simulated_samples().con1.plot()
else:
    qm = qmm.open_qm(config)
    job = qm.execute(rabi_amp_freq)
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")
    # Live plotting
    fig = plt.figure()
    plt.rcParams['figure.figsize'] = [12, 8]
    interrupt_on_close(fig, job)  #  Interrupts the job when closing the figure
    while results.is_processing():
        # Fetch results
        I, Q, iteration = results.fetch_all()
        # Progress bar
        progress_counter(iteration, n_avg, start_time=results.get_start_time())
        # Plot results
        plt.subplot(211)
        plt.cla()
        plt.title("Rabi chevron amplitude")
        plt.pcolor(dfs / u.MHz, amps * machine.qubits[qubit_index].pi_amp, np.sqrt(I**2 + Q**2))
        plt.xlabel("Frequency detuning [MHz]")
        plt.ylabel("Pulse amplitude [V]")
        plt.colorbar()
        plt.subplot(212)
        plt.cla()
        plt.title("Rabi chevron phase")
        plt.pcolor(dfs / u.MHz, amps * machine.qubits[qubit_index].pi_amp, signal.detrend(np.unwrap(np.angle(I + 1j * Q))))
        plt.xlabel("Frequency detuning [MHz]")
        plt.ylabel("Pulse amplitude [V]")
        plt.colorbar()
        plt.tight_layout()
        plt.pause(0.01)

plt.savefig(os.path.join(tPath, p_name))

2023-08-15 17:24:00,557 - qm - INFO     - Starting session: a4ffd96b-b8c7-4a8f-a36b-7632453c00ea
2023-08-15 17:24:13,939 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\qm1-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\qm1-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise se

C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:131: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:138: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:145: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:131: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)


C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:138: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:145: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:131: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:138: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:145: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:131: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:138: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:145: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


...

C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:131: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:138: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:145: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:131: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:138: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:145: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:131: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:138: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:145: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


Execution stopped by user!


C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:131: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:138: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)
C:\Users\painter\AppData\Local\Temp\ipykernel_28524\2051382538.py:145: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [ ]:
## convert everything to lists so they are easy to parse in Matlab if desired
Rabi_amp = amps * machine.qubits[qubit_index].pi_amp;
if isinstance(dfs,np.ndarray) == True:
   dfs1=dfs.tolist()
else:  
    pass
if isinstance(Rabi_amp,np.ndarray) == True:
   Rabi_amp1=Rabi_amp.tolist()
else:  
    pass

if isinstance(I,np.ndarray) == True:
   I1=I.tolist()
else:  
    pass
if isinstance(Q,np.ndarray) == True:
   Q1=Q.tolist()
else:  
    pass

data = [('Variable', 'Value'), ('Freq', dfs1), ('Amp', Rabi_amp1), ('I', I1), ('Q', Q1)]
df = pd.DataFrame(data,index=None, columns=None)
df.to_csv(os.path.join(tPath, f_name),header=False, index=False)
open(os.path.join(tPath, j_name), "w").write(open("quam_state.json").read())
#machine._save("quam_state.json", flat_data=False)